In [1]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))
    user: Mapped["User"] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

In [2]:
from sqlalchemy import create_engine
engine = create_engine("sqlite://", echo=True)

In [3]:
Base.metadata.create_all(engine)

2024-09-13 00:37:07,321 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-13 00:37:07,322 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2024-09-13 00:37:07,323 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-13 00:37:07,324 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2024-09-13 00:37:07,324 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-13 00:37:07,325 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2024-09-13 00:37:07,325 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-13 00:37:07,326 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2024-09-13 00:37:07,326 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-13 00:37:07,328 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30) NOT NULL, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2024-09-13 00:37:07,328 INFO sqlalchemy.engine.Engine [no key 0.00049s] ()
2024-09-13 00:37:07,330 INFO sqlalchemy.engine.

In [6]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )
    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )
    patrick = User(name="patrick", fullname="Patrick Star")
    session.add_all([spongebob, sandy, patrick])
    session.commit()

2024-08-19 08:56:07,254 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-19 08:56:07,255 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-08-19 08:56:07,256 INFO sqlalchemy.engine.Engine [cached since 159.7s ago (insertmanyvalues) 1/3 (ordered; batch not supported)] ('spongebob', 'Spongebob Squarepants')
2024-08-19 08:56:07,257 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-08-19 08:56:07,258 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/3 (ordered; batch not supported)] ('sandy', 'Sandy Cheeks')
2024-08-19 08:56:07,259 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-08-19 08:56:07,259 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/3 (ordered; batch not supported)] ('patrick', 'Patrick Star')
2024-08-19 08:56:07,262 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?) RETU

In [7]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name.in_(["spongebob", "sandy"]))

for user in session.scalars(stmt):
    print(user)

2024-08-19 08:56:11,280 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-19 08:56:11,284 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name IN (?, ?)
2024-08-19 08:56:11,286 INFO sqlalchemy.engine.Engine [generated in 0.00165s] ('spongebob', 'sandy')
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')


In [8]:
from sqlalchemy import select

session = Session(engine)

stmt = select(User).where(User.name == "patrick")

for user in session.scalars(stmt):
    print(user)

2024-08-19 08:56:13,623 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-19 08:56:13,626 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-08-19 08:56:13,627 INFO sqlalchemy.engine.Engine [generated in 0.00133s] ('patrick',)
User(id=3, name='patrick', fullname='Patrick Star')


In [9]:
stmt = (
    select(Address)
    .join(Address.user)
    .where(User.name == "sandy")
    .where(Address.email_address == "sandy@sqlalchemy.org")
)
sandy_address = session.scalars(stmt).one()
sandy_address

2024-08-19 08:56:15,608 INFO sqlalchemy.engine.Engine SELECT address.id, address.email_address, address.user_id 
FROM address JOIN user_account ON user_account.id = address.user_id 
WHERE user_account.name = ? AND address.email_address = ?
2024-08-19 08:56:15,608 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ('sandy', 'sandy@sqlalchemy.org')


Address(id=2, email_address='sandy@sqlalchemy.org')

In [10]:
stmt = select(User).where(User.name == "patrick")
patrick = session.scalars(stmt).one()
patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))
sandy_address.email_address = "sandy_cheeks@sqlalchemy.org"

session.commit()

2024-08-19 08:56:17,319 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-08-19 08:56:17,320 INFO sqlalchemy.engine.Engine [cached since 3.695s ago] ('patrick',)
2024-08-19 08:56:17,324 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-08-19 08:56:17,325 INFO sqlalchemy.engine.Engine [generated in 0.00109s] (3,)
2024-08-19 08:56:17,329 INFO sqlalchemy.engine.Engine UPDATE address SET email_address=? WHERE address.id = ?
2024-08-19 08:56:17,329 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ('sandy_cheeks@sqlalchemy.org', 2)
2024-08-19 08:56:17,331 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2024-08-19 08:56:17,331 INFO sqlalchemy.engine.Engine [generated in 0.00059s] ('patrickstar@sqlalchemy.org', 3

In [11]:
stmt = select(User)

In [12]:
for user in session.scalars(stmt):
    print(user)

2024-08-19 08:56:20,512 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-19 08:56:20,513 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2024-08-19 08:56:20,514 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ()
User(id=1, name='spongebob', fullname='Spongebob Squarepants')
User(id=2, name='sandy', fullname='Sandy Cheeks')
User(id=3, name='patrick', fullname='Patrick Star')


In [13]:
stmt = select(User).where(User.name == "patrick")

In [14]:
patrick = session.scalars(stmt).one()

2024-08-19 08:56:22,043 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-08-19 08:56:22,044 INFO sqlalchemy.engine.Engine [cached since 8.418s ago] ('patrick',)


In [15]:
patrick

User(id=3, name='patrick', fullname='Patrick Star')

In [16]:
patrick.addresses

2024-08-19 08:56:25,056 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-08-19 08:56:25,057 INFO sqlalchemy.engine.Engine [cached since 7.734s ago] (3,)


[Address(id=4, email_address='patrickstar@sqlalchemy.org')]

In [17]:
sandy = session.get(User, 2)
sandy.addresses.remove(sandy_address)

2024-08-19 08:56:26,906 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2024-08-19 08:56:26,907 INFO sqlalchemy.engine.Engine [generated in 0.00103s] (2,)
2024-08-19 08:56:26,909 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-08-19 08:56:26,910 INFO sqlalchemy.engine.Engine [cached since 9.586s ago] (2,)


In [18]:
session.flush()

2024-08-19 08:56:31,403 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2024-08-19 08:56:31,404 INFO sqlalchemy.engine.Engine [generated in 0.00112s] (2,)


In [19]:
session.delete(patrick)

In [20]:
session.commit()

2024-08-19 08:56:32,641 INFO sqlalchemy.engine.Engine DELETE FROM address WHERE address.id = ?
2024-08-19 08:56:32,642 INFO sqlalchemy.engine.Engine [cached since 1.24s ago] (4,)
2024-08-19 08:56:32,644 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2024-08-19 08:56:32,645 INFO sqlalchemy.engine.Engine [generated in 0.00086s] (3,)
2024-08-19 08:56:32,645 INFO sqlalchemy.engine.Engine COMMIT
